In [1]:
import jieba
import requests
from bs4 import BeautifulSoup
from textrank4zh import TextRank4Sentence
from wordcloud import WordCloud
# 请求的url


# url = 'https://3w.huanqiu.com/a/c36dc8/3xqGPRBcUE6?agt=8'
url = 'https://baijiahao.baidu.com/s?id=1690392756322616386'
html = requests.get(url, timeout=10)
content = html.content
# print(content)


In [2]:
# 通过content创建BeatifulSoup对象，将字符串格式的js转换成可解析的对象格式。
soup = BeautifulSoup(content, 'html.parser', from_encoding='utf-8')
text = soup.get_text()
# print(text)



In [3]:
import jieba.posseg as pseg

# 获取人物和地点
words = pseg.lcut(text)
# print(words)

news_person = {word for word, flag in words if flag == 'nr'}
news_place = {word for word, flag in words if flag == 'ns'}
# print(news_place)
# print(news_person)


Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Dracula\AppData\Local\Temp\jieba.cache
Loading model cost 0.632 seconds.
Prefix dict has been built successfully.


In [4]:
# 提取中文及相关标点
import re

text = re.sub('[^[\u4e00-\u9fa5。，！：、]{3,}', '', text)   # “\u4e00”和“\u9fa5”是unicode编码，并且正好是中文编码的开始和结束的两个值，这里的意思是除了中文字符和中文标点符号之外的字符全部用空字符串进行替换，重复进行3次以上
print(text)




哈尔滨市1月30日新增确诊病例5例 新增无症状感染者3例[[[[[[[];百度首页登录名字名字名哎个人中心帐号设置意见反馈退出哈尔滨市1月30日新增确诊病例5例 新增无症状感染者3例环球网发布时间：环球网官方帐号来源：央视新闻客户端年1月30日时，哈尔滨市新增新冠肺炎确诊病例5例。新增无症状感染者3例。上述所有确诊病例和无症状感染者均是在集中隔离医学观察期间及开展扩大核酸检测主动筛查发现的，经专家组会诊后明确诊断，现已转至定点医疗机构进行隔离治疗或隔离医学观察。有关部门已对所有排查出的密切接触者人和密接的密接全部落实隔离管控措施,对其曾经活动过的场所进行终末消毒和封闭管理。1月30日新增确诊病例主要情况：病例1：现住利民开发区乐业镇翻身屯病例2：现住利民开发区长禹星港湾病例3：现住呼兰区兰河街道富强村三队病例4：现住呼兰区兰河街道孟家乡红旗村病例5：现住道里区地德里小区以上新增5例新冠肺炎确诊病例，1月30日核酸结果为阳性，结合流行病学史、临床表现、实验室检测和影像学检查等结果，诊断为确诊病例。1月30日新增无症状感染者主要情况：无症状感染者1：现住呼兰区呼兰街道金街社区无症状感染者2：现住呼兰区第三发电厂家属楼无症状感染者3：现住呼兰区房产局楼新增3例无症状感染者，1月30日核酸结果为阳性，结合流行病学史、临床表现、实验室检测和影像学检查等结果，诊断为无症状感染者。截至1月30日24时，哈尔滨市现有新冠肺炎确诊病例及无症状感染者均在定点医院进行隔离治疗和隔离医学观察，生命体征平稳。1月9日至1月30日24时，累计追踪本次疫情密切接触者人，密切接触者的密切接触者人；现有集中隔离人，居家隔离人；累计采集核酸检测样本人，已出结果人，待出结果人，待检测人。（总台央视记者 张艺馨）举报/反馈作者最新文章河北石家庄：市区公交恢复运营线路条 占比网传 “石家庄市民做肛拭子核酸检测后秒变企鹅” 官方：系移花接木式谣言重磅微视频｜青山着意化为桥——习近平总书记关心北京冬奥会筹办工作纪实相关文章疫情期间怎么吃条饮食建议帮你提升免疫力不管有没有咳嗽，抗击病毒，都应该多吃点清肺、润肺、养肺的食物鱿鱼耳阳性！全部封存！快递是否会传播新型冠状病毒？1月30日江苏新增境外输入新冠肺炎确诊病例1例设为首页使用百度前必读 意见反馈 京证号 京公网安备号返回顶部请输入问题描述请填写联系方式


In [5]:
# 去掉停用词
def remove_stop_words(f):
    stop_words = ['不是']
    for stop_word in stop_words:
        f = f.replace(stop_word, '')
    return f


def create_word_cloud(f):
    f = remove_stop_words(f)
    seg_list = jieba.lcut(f)
    cut_text = ' '.join(seg_list)
    wc = WordCloud(
        max_words=100,
        width=2000,
        height=1200,
        font_path='./msyh.ttf'
    )
    wordcloud = wc.generate(cut_text)
    wordcloud.to_file("wordcloud.jpg")



In [7]:
create_word_cloud(text)

In [8]:
# 输出重要的句子
tr4s = TextRank4Sentence()
tr4s.analyze(text=text, lower=True, source = 'all_filters')
print('摘要：')
# 重要性较高的三个句子
for item in tr4s.get_key_sentences(num=3):
	# index是语句在文本中位置，weight表示权重
    print(item.index, item.weight, item.sentence)


摘要：
5 0.08773224347348513 1月30日新增确诊病例主要情况：病例1：现住利民开发区乐业镇翻身屯病例2：现住利民开发区长禹星港湾病例3：现住呼兰区兰河街道富强村三队病例4：现住呼兰区兰河街道孟家乡红旗村病例5：现住道里区地德里小区以上新增5例新冠肺炎确诊病例，1月30日核酸结果为阳性，结合流行病学史、临床表现、实验室检测和影像学检查等结果，诊断为确诊病例
3 0.08735846389350825 上述所有确诊病例和无症状感染者均是在集中隔离医学观察期间及开展扩大核酸检测主动筛查发现的，经专家组会诊后明确诊断，现已转至定点医疗机构进行隔离治疗或隔离医学观察
1 0.08336219686474586 百度首页登录名字名字名哎个人中心帐号设置意见反馈退出哈尔滨市1月30日新增确诊病例5例 新增无症状感染者3例环球网发布时间：环球网官方帐号来源：央视新闻客户端年1月30日时，哈尔滨市新增新冠肺炎确诊病例5例
